In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = loadOSCdata.downloadData(OSCid=367385, X = True, Y = True, Z = True, output = 'csv', outputFile = 'data.csv')
data.head()

,timestamp,point_id,v_value,geometry,image_url,image_lab,trip_id
792,2017-05-26 17:15:57.918,0.0,0.761113,POINT (-73.93671009000001 40.704303425),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
3462,2017-05-26 17:16:13.945,1.0,8.750670,POINT (-73.936764505 40.70428552),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
5727,2017-05-26 17:16:30.904,2.0,25.468589,POINT (-73.93671463999999 40.704234995),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
6128,2017-05-26 17:16:34.900,3.0,27.397556,POINT (-73.93687019000001 40.704219665),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
6434,2017-05-26 17:16:37.900,4.0,36.878148,POINT (-73.937003105 40.70423295000001),http://storage4.openstreetcam.org/files/photo/...,NaN,367385


In [3]:
bufers = gpd.read_file('../data/nyc-bike-routes/nyc_bike_30mbuffer_epsg=3857.shp')
bikelanes=gpd.read_file('../data/nyc-bike-routes/nyc_bike_routes_2017.shp')


In [4]:
pointsDF = loadOSCdata.snapToBikelane(bikelaneDF = bikelanes, bufersDF = bufers, pointsDF = data)
pointsDF.head()

,timestamp,point_id,v_value,geometry,image_url,image_lab,trip_id,line,bikelanesID,pointOnLine
792,2017-05-26 17:15:57.918000,0.0,0.761113,POINT (-8230596.918148693 4968823.837400841),http://storage4.openstreetcam.org/files/photo/...,NaN,367385,NaN,NaN,NaN
3462,2017-05-26 17:16:13.945000,1.0,8.750670,POINT (-8230602.975598783 4968821.208178158),http://storage4.openstreetcam.org/files/photo/...,NaN,367385,NaN,NaN,NaN
5727,2017-05-26 17:16:30.904000,2.0,25.468589,POINT (-8230597.424652374 4968813.788942861),http://storage4.openstreetcam.org/files/photo/...,NaN,367385,NaN,NaN,NaN
6128,2017-05-26 17:16:34.900000,3.0,27.397556,POINT (-8230614.740399169 4968811.537842993),http://storage4.openstreetcam.org/files/photo/...,NaN,367385,NaN,NaN,NaN
6434,2017-05-26 17:16:37.900000,4.0,36.878148,POINT (-8230629.536429288 4968813.488649339),http://storage4.openstreetcam.org/files/photo/...,NaN,367385,NaN,NaN,NaN


In [16]:
pointsDF.timestamp = pointsDF.timestamp.map(lambda x: str(x))

point = pointsDF.drop(['line','pointOnLine'],axis=1)
point = point.to_crs(epsg=4326)
point.to_file('point')

In [48]:
line = pointsDF.drop(['geometry','pointOnLine','image_lab'],axis=1)


line.columns = [u'timestamp', u'point_id', u'v_value', u'image_url', 
       u'trip_id', u'geometry', u'bikelanesID']
line = line.dropna().to_crs(epsg=4326)
line.to_file('line')


In [53]:
newPoint = pointsDF.drop(['geometry','line','image_lab'],axis=1)
newPoint.columns = [u'timestamp', u'point_id', u'v_value', u'image_url', u'trip_id',
       u'bikelanesID', u'geometry']
newPoint = newPoint.dropna().to_crs(epsg=4326)
newPoint.to_file('newPoint')
newPoint.head()

,timestamp,point_id,v_value,image_url,trip_id,bikelanesID,geometry
42377,2017-05-26 17:22:30.899000,128.0,3.377756,http://storage4.openstreetcam.org/files/photo/...,367385,1712.0,POINT (-73.94377553741641 40.71165429059842)
43702,2017-05-26 17:22:43.893000,129.0,3.983192,http://storage4.openstreetcam.org/files/photo/...,367385,1712.0,POINT (-73.94382206436049 40.71164989128519)
46824,2017-05-26 17:23:14.900000,130.0,67.564336,http://storage4.openstreetcam.org/files/photo/...,367385,1712.0,POINT (-73.94390647442297 40.71164190996713)
47234,2017-05-26 17:23:18.877000,131.0,45.602697,http://storage4.openstreetcam.org/files/photo/...,367385,1711.0,POINT (-73.94413045096925 40.71162064934354)
48154,2017-05-26 17:23:27.884000,135.0,36.302540,http://storage4.openstreetcam.org/files/photo/...,367385,1711.0,POINT (-73.94482475966959 40.71155473789594)
